<a href="https://colab.research.google.com/github/raleight1/mgmt467-analytics-portfolio/blob/main/Labs/Labs%201-3/Lab1_AI_Assisted_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 1: AI‑Assisted SQL Foundations (Colab)**
**Course:** MGMT 467 — AI‑Assisted Big Data Analytics in the Cloud  
**When:** Unit 1 · Week 3 (Thursday)  
**Goal:** Use *Gemini as a co‑pilot* to write and understand basic SQL against the **Superstore** dataset in **BigQuery**.  
**You will practice:** Prompt engineering → SQL generation → Query execution → Interpretation.

> ✅ **Deliverable (submit link in Brightspace):** A completed `Lab1_AI_Assisted_SQL.ipynb` with prompts, code, outputs.


## ✅ What you need before starting
- A Google account with access to **Google Colab**.
- A **Google Cloud Project** with **BigQuery** API enabled.
- A dataset named `superstore_data` and a table named `sales` (your instructor or TA will provide details).
- (Optional) A GitHub repo to store your work for your team.


## 0) Connect to class GitHub repo
## Install and import libraries
## Class Repo: bigDataNCloud/mgmt_467_resources

In [ ]:
# # Replace with your actual class repo URL
# CLASS_REPO_URL = "https://github.com/YOUR-ORG-OR-USERNAME/class-repo.git"
# CLASS_REPO_DIR = "/content/class-repo"

# # If already cloned, skip re-downloading
# import os
# if not os.path.exists(CLASS_REPO_DIR):
#     !git clone {CLASS_REPO_URL} {CLASS_REPO_DIR}
# else:
#     %cd {CLASS_REPO_DIR}
#     !git pull

# # Move into repo folder
# %cd {CLASS_REPO_DIR}

# print("✅ Class repository ready at", CLASS_REPO_DIR)

In [ ]:
!pip -q install google-cloud-bigquery google-cloud-bigquery-storage db-dtypes pandas pyarrow kagglehub

In [ ]:
from google.cloud import bigquery
from google.colab import auth
import pandas as pd
import os, json, textwrap, pathlib, pprint
print("Libraries imported.")

Libraries imported.


## 1) Authenticate to GCP and set your project
Run the cell below. When prompted, authorize Colab to access your Google Cloud resources.

In [ ]:
auth.authenticate_user()
print("✅ Authenticated.")

# Set your GCP project (EDIT THIS!)
PROJECT_ID = "mgmt-467-lab-1-ai-assisted-sql"   # <-- EDIT ME
assert PROJECT_ID != "YOUR_PROJECT_ID_HERE", "Please set PROJECT_ID to your GCP project id."
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

# BigQuery client
bq = bigquery.Client(project=PROJECT_ID)
print("✅ BigQuery client created for:", PROJECT_ID)

✅ Authenticated.
✅ BigQuery client created for: mgmt-467-lab-1-ai-assisted-sql


### Quick connection test (optional)
This will try to read the first 5 rows from your Superstore table. If it fails, double‑check your dataset/table name.

In [ ]:
DATASET = "lab1_foundation"
TABLE   = "superstore"
BQ_TABLE = f"`{PROJECT_ID}.{DATASET}.{TABLE}`"

query = f"SELECT * FROM {BQ_TABLE} LIMIT 5"
try:
    df_preview = bq.query(query).to_dataframe()
    display(df_preview)
    print("✅ Connection OK.")
except Exception as e:
    print("⚠️ Could not query the table. Error below:")
    print(e)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,5769,CA-2015-154900,2015-02-25,2015-03-01,Standard Class,SS-20875,Sung Shariari,Consumer,United States,Leominster,...,1453,East,OFF-LA-10001641,Office Supplies,Labels,Avery 518,3.15,1,0.0,1.5120
1,5770,CA-2015-154900,2015-02-25,2015-03-01,Standard Class,SS-20875,Sung Shariari,Consumer,United States,Leominster,...,1453,East,OFF-PA-10002377,Office Supplies,Paper,Adams Telephone Message Book W/Dividers/Space ...,22.72,4,0.0,10.2240
2,9028,US-2016-152415,2016-09-17,2016-09-22,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,Marlborough,...,1752,East,FUR-FU-10002597,Furniture,Furnishings,"C-Line Magnetic Cubicle Keepers, Clear Polypro...",14.82,3,0.0,6.2244
3,9029,US-2016-152415,2016-09-17,2016-09-22,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,Marlborough,...,1752,East,FUR-FU-10004864,Furniture,Furnishings,"Howard Miller 14-1/2"" Diameter Chrome Round Wa...",191.82,3,0.0,61.3824
4,8332,CA-2016-153269,2016-03-09,2016-03-12,First Class,PS-18760,Pamela Stobb,Consumer,United States,Andover,...,1810,East,OFF-ST-10004634,Office Supplies,Storage,"Personal Folder Holder, Ebony",11.21,1,0.0,3.3630


✅ Connection OK.


## 2) (Optional) Download a dataset with **KaggleHub**
Use **KaggleHub** to pull sample data locally into Colab (e.g., for offline exploration or to stage for GCS).
> This is **optional** for Lab 1 (we focus on BigQuery), but useful to practice data access workflows.
> https://www.kaggle.com/datasets/anandaramg/global-superstore

In [ ]:
import kagglehub, os, pathlib

DATASET_REF = "anandaramg/global-superstore"   # you can change this to any public dataset reference

download_path = kagglehub.dataset_download(DATASET_REF)
print("Path to dataset files:", download_path)

# List files
p = pathlib.Path(download_path)
print("Files:")
for f in p.glob("**/*"):
    if f.is_file():
        print("-", f)


100%|██████████| 3.18M/3.18M [00:00<00:00, 141MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/anandaramg/global-superstore/versions/1
Files:
- /root/.cache/kagglehub/datasets/anandaramg/global-superstore/versions/1/Global Superstore.txt


## 3) Prompting approach for Lab 1
You'll **paste prompts into Gemini** (in a separate tab), get back **SQL**, then paste that SQL here to execute.  
We’ll practice three core question types:
- **“What”** (lists & filters) → `SELECT`, `WHERE`, `DISTINCT`
- **“How many”** (counts by category) → `COUNT`, `GROUP BY`
- **“Who is best”** (rank/limit by metric) → `SUM`, `ORDER BY`, `LIMIT`

### 3A) “What” Question — `SELECT`, `WHERE`, `DISTINCT`
**Business Question:** *A manager wants a list of all **unique** product sub‑categories sold in the **West** region.*

**Paste this prompt into Gemini (no edits needed except the project id if you used a different table path):**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`.
# GOAL: Find all the unique values in the 'Sub_Category' column, but only for rows where the 'Region' column is exactly 'West'.
```

**Then paste the SQL you get into the cell below** (replace the placeholder) and run it.

In [ ]:
sql_str = """
SELECT
  DISTINCT `Sub-Category`
FROM
  `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore`
WHERE
  Region = 'West'
"""
sql_str = sql_str.replace("mgmt-467-lab-1-ai-assisted-sql", PROJECT_ID)

df_what = bq.query(sql_str).to_dataframe()
print(f"Rows: {len(df_what)}")
display(df_what.head(20))

Rows: 17


,Sub-Category
0,Storage
1,Phones
2,Accessories
3,Binders
4,Furnishings
5,Appliances
6,Envelopes
7,Fasteners
8,Bookcases
9,Chairs


**Explain it back (metacognition):** In Gemini, paste:  
> *Explain the following SQL query line by line: [paste your SQL]*  
Summarize what you learned here:

> **Notes:** Select distinct ensures that there will be no duplicates when selecting from the Sub-Category column. From specifies the table to get the information from. Where filters so that the results only include rows where the region column has 'West'.

### 3B) “How many” Question — `COUNT`, `GROUP BY`
**Business Question:** *How many orders were placed in each **Ship Mode**?*

**Gemini prompt:**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`.
# GOAL: Count the total number of records for each unique value in the 'Ship_Mode' column.
# The final result should have two columns: 'Ship_Mode' and 'order_count'.
```


In [ ]:
sql_str = """
SELECT `Ship Mode`, COUNT(*) AS order_count
FROM `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore`
GROUP BY `Ship Mode`
ORDER BY order_count DESC
"""
sql_str = sql_str.replace("mgmt-467-lab-1-ai-assisted-sql", PROJECT_ID)

df_howmany = bq.query(sql_str).to_dataframe()
display(df_howmany)

,Ship Mode,order_count
0,Standard Class,5968
1,Second Class,1945
2,First Class,1538
3,Same Day,543


### 3C) “Who is best” Question — `SUM`, `ORDER BY`, `LIMIT`
**Business Question:** *Identify the **top 5 most profitable customers**.*

**Gemini prompt:**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`. The customer identifier is 'Customer_ID'.
# GOAL: Calculate the sum of 'Profit' for each customer. The final output should show the 'Customer_ID' and their total profit, sorted from highest to lowest profit, and limited to only the top 5 results.
```


In [ ]:
sql_str = """
-- PASTE the SQL from Gemini here
SELECT `Customer ID`, SUM(Profit) AS total_profit
FROM `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore`
GROUP BY `Customer ID`
ORDER BY total_profit DESC
LIMIT 5
"""
sql_str = sql_str.replace("mgmt-467-lab-1-ai-assisted-sql", PROJECT_ID)

df_best = bq.query(sql_str).to_dataframe()
display(df_best)

,Customer ID,total_profit
0,TC-20980,8981.3239
1,RB-19360,6976.0959
2,SC-20095,5757.4119
3,HL-15040,5622.4292
4,AB-10105,5444.8055


## 4) Challenge prompts (author your own)
Write **your own precise prompt** in a text cell (or comment) for each question below, then get SQL from Gemini and run it in the provided code cells.

**Challenge 1:** *What is the **average discount** for products in the **Technology** category sold in the **East** region?*  
**Challenge 2:** *How many **unique customers** has each **Segment** (Consumer/Corporate/Home Office) served?*

Task: Generate a BigQuery SQL query.

Context: The table is "mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore". You will utilize the "Discount", "Category", and "Region" columns.

Goal: calculate the Average Discount for products in the Technology Category sold in the East Region.

In [ ]:
sql_str = """
SELECT Category, AVG(Discount) AS avg_discount
FROM `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore`
WHERE Category = 'Technology' AND Region = 'East'
GROUP BY Category
"""
sql_str = sql_str.replace("mgmt-467-lab-1-ai-assisted-sql", PROJECT_ID)
df_ch1 = bq.query(sql_str).to_dataframe()
display(df_ch1)

,Category,avg_discount
0,Technology,0.143364


Task: Generate a BigQuery SQL query.

Context: the table is "mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore". Utilizing the "Customer ID", and "Segment".

Goal: Determine how many unique customers each segment (Consumer, Corporate, Home office) served?

In [ ]:
sql_str = """
SELECT
    Segment,
    COUNT(DISTINCT `Customer ID`) AS unique_customers
FROM
    `mgmt-467-lab-1-ai-assisted-sql.lab1_foundation.superstore`
GROUP BY
    Segment
ORDER BY
    unique_customers DESC
"""

df_ch2 = bq.query(sql_str).to_dataframe()
display(df_ch2)

,Segment,unique_customers
0,Consumer,409
1,Corporate,236
2,Home Office,148


## 5) Reflection (DIVE mindset)
- **Discover:** What did you find first?  
- **Investigate:** What alternate query or filter changed the story?  
- **Validate:** Where could the AI‑generated SQL be wrong or incomplete? How did you check?  
- **Extend:** Which stakeholder could use your results tomorrow? What action should they take?

The first thing that I noticed in the exploration of the superstore dataset utilizing AI as a copilot was the ease that we were able to generate at least the correct foundation for a successful query and how adding more context made it so the queries were correct right away. Looking deeper into the profit by top 5 customers was an interesting query that introduced an understanding that there is a large gap between the top 2 customers and the rest regarding the profit coming from these groups. The AI generated SQL struggled the most when it came to not recieving enough context regarding specific column names and ensuring the these names were included in the prompt helped significantly with error management. My understanding of the data schema itself helped alot in this regard because it allowed me to immediately spot where the errors were occuring. This very basic information alone could be beneficial to the operations division of the business to understand where they are generating the most profit and seeing what strategies with individual customers are the most effective.

## 6) Save your work to GitHub (pick one of the options)
**Option A (recommended):** In Colab, go to **File → Save a copy in GitHub…** and select your team repo + folder (e.g., `labs/Unit1/`).  
**Option B (CLI, if you know git):**
```bash
# (In Colab) mount Drive, then clone/pull/push as usual with a PAT
# Be careful to NOT store secrets in the notebook.
```
Name the file **`Lab1_AI_Assisted_SQL.ipynb`** and push it to your team repo.